## Камбаров Динмухаммед

Реализуйте сортировки: пузырьковую, пирамидальную и быструю.
 
Каждую сначала на чистом Python, а потом ускоряя как с помощью распараллеливания, так и jit-компиляторов.
 
Оцените зависимость времени выполнения от размера массива данных.
 
Проведите сравнение времени выполнения на одинаковых наборах данных.

In [1]:
import numpy as np
from numba import jit
import numba as nb
import os
from numba.typed import List
from threading import Thread
import threading
import time
import random, time, sys
from multiprocessing import Pool, Process, Pipe, cpu_count
from itertools import chain
import pandas as pd 

MAX_NUM = 1000000


In [2]:
def is_sorted(X):
    if len(X) <= 1:
        return True
    for i in range(1, len(X)):
        if X[i] < X[i - 1]:
            return False
    return True

In [3]:
def generate_random_array(N):
    return [random.randint(0, MAX_NUM) for i in range(N)]

# Bubble Sort

In [4]:
def bubblesort(X):
    n = len(X)
    for i in range(n - 1):
        for j in range(i + 1, n):
            if X[i] > X[j]:
                X[i], X[j] = X[j], X[i]
    return X
        

In [5]:
@jit(nopython = True)
def bubblesort_jit(X):
    n = len(X)
    for i in range(n - 1):
        for j in range(i + 1, n):
            if X[i] > X[j]:
                X[i], X[j] = X[j], X[i]
    return X

#### К сожалению, я не смог найти способ распараллелить пузырьковую сортировку

# Quick Sort

In [6]:
def partition(arr, low, high):
    i = (low-1)         # index of smaller element
    pivot = arr[high]     # pivot
  
    for j in range(low, high):
  
        # If current element is smaller than or
        # equal to pivot
        if arr[j] <= pivot:
  
            # increment index of smaller element
            i = i+1
            arr[i], arr[j] = arr[j], arr[i]
  
    arr[i+1], arr[high] = arr[high], arr[i+1]
    return (i+1)

def qsort(arr, low, high):
    if len(arr) == 1:
        return arr
    if low < high:
  
        # pi is partitioning index, arr[p] is now
        # at right place
        pi = partition(arr, low, high)
  
        # Separately sort elements before
        # partition and after partition
        qsort(arr, low, pi-1)
        qsort(arr, pi+1, high)
    return arr

def quicksort(arr):
    return qsort(arr, 0, len(arr) - 1)

In [7]:
@jit( nopython=True )
def partition_jit(arr, low, high):
    i = (low-1)         # index of smaller element
    pivot = arr[high]     # pivot
  
    for j in range(low, high):
  
        # If current element is smaller than or
        # equal to pivot
        if arr[j] <= pivot:
  
            # increment index of smaller element
            i = i+1
            arr[i], arr[j] = arr[j], arr[i]
  
    arr[i+1], arr[high] = arr[high], arr[i+1]
    return (i+1)

@jit(nopython=True)
def qsort_jit(arr, low, high):
    if len(arr) == 1:
        return arr
    if low < high:
  
        # pi is partitioning index, arr[p] is now
        # at right place
        pi = partition_jit(arr, low, high)
  
        # Separately sort elements before
        # partition and after partition
        qsort_jit(arr, low, pi-1)
        qsort_jit(arr, pi+1, high)
    return arr
    
@jit(nopython=True) 
def quicksort_jit(arr):
    return qsort_jit(arr, 0, len(arr) - 1)

In [8]:
def qsort_parallel(X,left,right):
    i = left
    j = right
    pivot = X[(left + right)//2]
    temp = 0
    while(i <= j):
         while(pivot > X[i]):
             i += 1
         while(pivot < X[j]):
             j -= 1
         if(i <= j):
             X[i], X[j] = X[j], X[i]
             i += 1
             j -= 1

    lthread = None
    rthread = None

    if (left < j):
        lthread = Thread(target = lambda: qsort_parallel(X,left,j))
        lthread.start()

    if (i < right):
        rthread = Thread(target=lambda: qsort_parallel(X,i,right))
        rthread.start()

    if lthread is not None: lthread.join()
    if rthread is not None: rthread.join()
    return X

def quicksort_parallel(arr):
    return qsort_parallel(arr, 0, len(arr) - 1)

# Heap Sort

In [9]:
def heapify(X, n, i):
    largest = i
    l = 2 * i + 1     # left = 2*i + 1
    r = 2 * i + 2     # right = 2*i + 2
  
    if l < n and X[i] < X[l]:
        largest = l
  
    if r < n and X[largest] < X[r]:
        largest = r
  
    if largest != i:
        X[i], X[largest] = X[largest], X[i]
        heapify(X, n, largest)
        
def heap_sort(X):
    n = len(X)
    # Build a maxheap.
    # Since last parent will be at ((n//2)-1) we can start at that location.
    for i in range(n // 2 - 1, -1, -1):
        heapify(X, n, i)
  
    # One by one extract elements
    for i in range(n-1, 0, -1):
        X[i], X[0] = X[0], X[i]   # swap
        heapify(X, i, 0)
    return X

In [10]:
@jit(nopython=True)
def heapify_jit(X, n, i):
    largest = i
    l = 2 * i + 1     # left = 2*i + 1
    r = 2 * i + 2     # right = 2*i + 2
  
    if l < n and X[i] < X[l]:
        largest = l
  
    if r < n and X[largest] < X[r]:
        largest = r
  
    if largest != i:
        X[i], X[largest] = X[largest], X[i]
        heapify_jit(X, n, largest)

@jit(nopython=True)  
def heap_sort_jit(X):
    n = len(X)
  
    # Build a maxheap.
    for i in range(n // 2 - 1, -1, -1):
        heapify_jit(X, n, i)
    # Extract to the end of the array and heapify
    for i in range(n-1, 0, -1):
        X[i], X[0] = X[0], X[i]   
        heapify_jit(X, i, 0)
    return X

In [11]:
def merge(arrs):
    n = len(arrs)
    pointers = [0] * n
    ans = []
    while True:
        min_ind, min_elem = -1, MAX_NUM
        for i, pointer in enumerate(pointers):
            if pointer < len(arrs[i]) and min_elem > arrs[i][pointer]:
                min_elem, min_ind = arrs[i][pointer], i
        if min_ind == -1:
            break
        ans.append(min_elem)
        pointers[min_ind] += 1
    return ans

In [12]:
import hsort
def heapsort_parallel(X, num_procs = cpu_count()):
    n = len(X)
    part, r = divmod(n, num_procs)
    out = None
    with Pool(processes=num_procs) as pool:
            sub_arrs = [X[part * id : part * (id + 1)] for id in range(num_procs)]
            if r != 0:
                sub_arrs.append(X[n - r : n])
            return merge(pool.map(func=hsort.heap_sort, iterable=sub_arrs))

# Compare algorithms

In [13]:
arr_sizes = [100, 1000, 10000]

In [14]:
algos = {
    "bubblesort" : [bubblesort, bubblesort_jit],
    "heapsort"   : [heap_sort, heap_sort_jit, heapsort_parallel],
    "quicksort"  : [quicksort, quicksort_jit, quicksort_parallel]
}

In [15]:
import time
def measure_time(sort_fun, arr):
    start = time.time()
    arr = sort_fun(arr)
    end = time.time()
    print("SORTED" if is_sorted(arr) else "NOT SORTED")
    print(f">>> Execution time of {sort_fun.__name__}: {end - start} s")
    return (end - start) * 1000

In [16]:
results = {
    "bubblesort" : [[] for i in range(len(arr_sizes))],
    "heapsort"   : [[] for i in range(len(arr_sizes))],
    "quicksort"  : [[] for i in range(len(arr_sizes))],
}

In [17]:
for ind, arr_size in enumerate(arr_sizes):
    arr = generate_random_array(arr_size)
    for fun_name in algos.keys():
        for sort_fun in algos[fun_name]:
            arr_cp = arr.copy()
            if 'jit' in sort_fun.__name__:
                arr_list = List()
                for x in arr:
                    arr_list.append(x)
                res = measure_time(sort_fun, arr_list)
            else:
                res = measure_time(sort_fun, arr_cp)
            results[fun_name][ind].append(res)


SORTED
>>> Execution time of bubblesort: 0.0008492469787597656 s
SORTED
>>> Execution time of bubblesort_jit: 0.3081169128417969 s
SORTED
>>> Execution time of heap_sort: 0.0002560615539550781 s
SORTED
>>> Execution time of heap_sort_jit: 0.43648672103881836 s
SORTED
>>> Execution time of heapsort_parallel: 0.15256595611572266 s
SORTED
>>> Execution time of quicksort: 0.00014901161193847656 s
SORTED
>>> Execution time of quicksort_jit: 0.7494900226593018 s
SORTED
>>> Execution time of quicksort_parallel: 0.009073257446289062 s
SORTED
>>> Execution time of bubblesort: 0.061316728591918945 s
SORTED
>>> Execution time of bubblesort_jit: 0.032083988189697266 s
SORTED
>>> Execution time of heap_sort: 0.0038309097290039062 s
SORTED
>>> Execution time of heap_sort_jit: 0.001650094985961914 s
SORTED
>>> Execution time of heapsort_parallel: 0.14925599098205566 s
SORTED
>>> Execution time of quicksort: 0.0022079944610595703 s
SORTED
>>> Execution time of quicksort_jit: 0.0007338523864746094 s
SO

In [18]:
cols = ['bubblesort', 'bubblesort JIT', 'heapsort', 'heapsort JIT', 'heapsort PARALLEL', 'quicksort', 'quicksort JIT', 'quicksort PARALLEL']

df = pd.DataFrame(columns=cols, index=arr_sizes)

In [19]:
names = {
    'bubblesort' : ['bubblesort', 'bubblesort JIT'],
    'heapsort'   : ['heapsort', 'heapsort JIT', 'heapsort PARALLEL'],
    'quicksort'  : ['quicksort', 'quicksort JIT', 'quicksort PARALLEL']
}

In [20]:
for fun_name in results.keys():
    i = 0
    sort_names = names[fun_name]
    for i, result in enumerate(results[fun_name]):
        for j, res in enumerate(result):
            df[sort_names[j]][arr_sizes[i]] = res

#### Для наглядности время возвращаемое из функции measure time умноженно на 1000

In [21]:
df

,bubblesort,bubblesort JIT,heapsort,heapsort JIT,heapsort PARALLEL,quicksort,quicksort JIT,quicksort PARALLEL
100,0.849247,308.116913,0.256062,436.486721,152.565956,0.149012,749.490023,9.073257
1000,61.316729,32.083988,3.83091,1.650095,149.255991,2.207994,0.733852,78.431129
10000,5979.614019,3243.671894,80.300808,20.121098,260.020018,36.319971,10.64086,15373.176813


In [22]:
df.agg(['min', 'max'])


,bubblesort,bubblesort JIT,heapsort,heapsort JIT,heapsort PARALLEL,quicksort,quicksort JIT,quicksort PARALLEL
min,0.849247,32.083988,0.256062,1.650095,149.255991,0.149012,0.733852,9.073257
max,5979.614019,3243.671894,80.300808,436.486721,260.020018,36.319971,749.490023,15373.176813


In [23]:
df.assign(min_value=df.values.min(1))



,bubblesort,bubblesort JIT,heapsort,heapsort JIT,heapsort PARALLEL,quicksort,quicksort JIT,quicksort PARALLEL,min_value
100,0.849247,308.116913,0.256062,436.486721,152.565956,0.149012,749.490023,9.073257,0.149012
1000,61.316729,32.083988,3.83091,1.650095,149.255991,2.207994,0.733852,78.431129,0.733852
10000,5979.614019,3243.671894,80.300808,20.121098,260.020018,36.319971,10.64086,15373.176813,10.64086


#### Как видно из результатов, лучше всего себя показал алгоритм  quicksort_jit